In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from refinegems.classes.gapfill import KEGGapFiller
from refinegems.utility.io import load_model

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [3]:
mpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JCSC1435.xml'

model = load_model(mpath,'libsbml')

In [4]:
gff2 = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JCSC1435_RefSeq.gff'
missing_genes = get_missing_genes(gff2,load_model('/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JCSC1435.xml','libsbml'))
missing_genes

# @TODO : some od the genes do not have an old locus tag -> what to do?

NameError: name 'get_missing_genes' is not defined

In [2]:
from refinegems.classes.gapfill import GeneGapFiller
from refinegems.utility.io import load_model

modelpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435.xml'
model = load_model(modelpath,'libsbml')
cmodel = load_model(modelpath,'cobra')

gffpath = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435_RefSeq.gff'

gf2 = GeneGapFiller()
gf2_missing_genes = gf2.get_missing_genes(gffpath,model)
# ncbiprotein | locus_tag | ec-code

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [3]:
gf2_missing_genes

,ncbiprotein,locus_tag,ec-code
0,WP_011274359.1,SH0001,None
1,WP_011274360.1,SH0002,None
2,WP_172458781.1,SH0003,None
3,WP_011274362.1,SH0004,None
4,WP_011274363.1,SH0005,None
...,...,...,...
2573,WP_145424740.1,pSHaeC05,None
2574,WP_011276926.1,pSHaeC06,None
2575,WP_011276927.1,pSHaeC07,None
2576,WP_011276928.1,pSHaeC08,None


In [34]:
import pandas as pd
from refinegems.classes.gapfill import map_ec_to_reac
from refinegems.curation.db_access.db import get_ec_from_ncbi, get_ec_via_swissprot
# for notebook, later should be part of object
manual_curation = {}

# find missing reaction
# @TODO one fildering step towards the end is still missing
def get_missing_reacs(missing_genes, 
                      # NCBI params
                      mail:str=None, check_NCBI:bool=False,
                      # SwissProt
                      fasta:str=None, dmnd_db:str=None, swissprot_map:str=None,
                      **kwargs):
    
    # Case 1:  no EC
    # --------------
    case_1 = missing_genes[missing_genes['ec-code'].isna()]
    not_case_1 = missing_genes[~missing_genes['ec-code'].isna()]
    if len(case_1) > 0:
        
        # Option 1: BLAST against SwissProt
        # +++++++++++++++++++++++++++++++++    
        # -> BLAST (DIAMOND) against SwissProt to get EC/BRENDA 
        # @TEST
        if fasta and dmnd_db and swissprot_map:
            case_1_mapped = get_ec_via_swissprot(fasta,dmnd_db,
                                          case_1,
                                          swissprot_map,
                                          **kwargs) # further optional params for the mapping
            case_1.drop('ec-code', inplace=True, axis=1)
            case_1 = case_1.merge(case_1_mapped, on='locus_tag', how='left')
            not_case_1['UniProt'] = None
            
        # Option 2: Use ML to predict EC
        # ++++++++++++++++++++++++++++++
        # @TODO
        # -> sth like DeepECTransformer (tools either not good, 
        #    not installable or no license)
        # -> use ECRECer Web service output as input 
        #    (whole protein fasta -> wait for Gwendolyn's results) -> does not work / not return
        # -> use CLEAN webservice
        #    same problem as above with the web tool

    manual_curation['no ncbiprotein, no EC'] = case_1[case_1['ncbiprotein'].isna() & case_1['ec-code'].isna()] 
    
    mapped_reacs = pd.concat([case_1,not_case_1])

    # Case 2: still no EC but ncbiprotein
    # -----------------------------------
    #       -> access ncbi for ec (optional) 
    if check_NCBI and mail:
        mapped_reacs['ec-code'] = mapped_reacs.progress_apply(lambda x: get_ec_from_ncbi(mail,x['ncbiprotein']) if not x['ec-code'] else x['ec-code'], axis=1)
    
    manual_curation['no EC'] = mapped_reacs[mapped_reacs['ec-code'].isna()]
    mapped_reacs = mapped_reacs[~mapped_reacs['ec-code'].isna()]

    # Case 3: EC found
    # ----------------
    # @TODO : reformatting of table to ensure mapping each EC as less as possible 
    #         (ideally only once)
    # @TODO : how to carry the uniprot information along
    #       -> directly access reaction 
    mapped_reacs = map_ec_to_reac(mapped_reacs[['ec-code','ncbiprotein']])
    
    return mapped_reacs

In [35]:
tfasta = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435/JCSC1435_proteins_genome.fasta'
spdb = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/swissprot.dmnd'
# gf2_missing_genes
spmap = '/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/uniprot_table.tsv'
kwargs = {'outdir':'/Users/brune/Documents/11_Test_Data/test_refinegems/test_gapfill/JSC1435',
          'sens':'more-sensitive',
          'cov':90.0,
          't':4,
          'pid':90.0}

mapped_res = get_missing_reacs(missing_genes=gf2_missing_genes,
                               fasta=tfasta, 
                               dmnd_db=spdb,
                               swissprot_map=spmap,
                               **kwargs)

In [36]:
mapped_res

,ncbiprotein,locus_tag,ec-code,UniProt
0,WP_011274359.1,SH0001,NaN,NaN
1,WP_011274360.1,SH0002,NaN,NaN
2,WP_172458781.1,SH0003,NaN,NaN
3,WP_011274362.1,SH0004,NaN,NaN
4,WP_011274363.1,SH0005,5.6.2.2,"[Q5HK03, Q8CQK4, Q6GKU0, Q2FKQ1, Q6GD85, P0A0K..."
...,...,...,...,...
1698,WP_145424740.1,pSHaeC05,NaN,NaN
1699,WP_011276926.1,pSHaeC06,2.3.2.22,[Q4L2X9]
1700,WP_011276927.1,pSHaeC07,NaN,NaN
1701,WP_011276928.1,pSHaeC08,NaN,NaN


In [ ]:
# @TODO
# SwissProt Download
# (mappings + sequences can be downloaded from DB)
# two tables: 
#   1.: TSV with UniprotID, BRENDA and EC -7.7MB (26.07.2024)
#   2.: FASTA with sequences ~280MB (26.07.2024)
# more infos:
#   - API generation possible for automatic download
#   - maybe write a small script to download stuff and shorten to actual EC/BRENDA mappings

# current_release or complete? differences?
swissprot_api = 'https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz'
swissprot_mapping_api = 'https://rest.uniprot.org/uniprotkb/stream?compressed=true&fields=accession%2Cxref_brenda%2Cec%2Csequence&format=tsv&query=%28*%29+AND+%28reviewed%3Atrue%29'

def download_swissprot_for_gapfill():
    pass